## Load packages

In [ ]:
import numpy as np
import pylab as plt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import colorcet
import umap
import scanpy as sc
import pickle
import time
import itertools
import warnings

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score

## Load data

In [ ]:
dataset = sc.read_h5ad(DATA_DIR /"chu_2016.h5ad")

In [ ]:
dataset

In [ ]:
data = dataset.X.toarray()
labels = dataset.obs.labels

In [ ]:
pd.DataFrame(labels).value_counts()

In [ ]:
labels_copy = labels.copy()

In [ ]:
le = LabelEncoder()
le.fit(labels)
labels = le.transform(labels)

## Normalization of data

In [ ]:
from scipy import stats

#z-score (normalization on cell level)
data = stats.zscore(data, axis=1)

## Nested cross-validation

### Nested cross-validation on unreduced dataset

In [ ]:
#nested cross-validation
#in: X (array), y (pd Series), classifier (string) , param_grid (dict)
#out: yhat (pd Series)

def nested_cv(X, y, classifier, param_grid, outer_folds=3, inner_folds=3):
    
    #create empty list for test index and yhat
    ind = [] 
    yhat_list = []
    count = 0
    
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=1)
    
    for train_ind, test_ind in outer_cv.split(X):
        
        #split data into train and test
        X_train, X_test = X[train_ind, :], X[test_ind, :]
        y_train, y_test = y[train_ind], y[test_ind]
        
        #select classifier
        if classifier == "svm":
            model_to_tune = SVC(random_state=1)
        if classifier == "knn":
            model_to_tune = KNeighborsClassifier()
        if classifier == "rf":
            model_to_tune = RandomForestClassifier(random_state=1)
        if classifier == "mlr":
            model_to_tune = LogisticRegression(random_state=1)
        if classifier == "lda":
            model_to_tune = LinearDiscriminantAnalysis()
        if classifier == "qda":
            model_to_tune = QuadraticDiscriminantAnalysis()

        #inner cross-validation
        inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=1)
        grid = GridSearchCV(model_to_tune, param_grid, cv=inner_cv, refit=True, n_jobs=-1, scoring='accuracy')
        result = grid.fit(X_train, y_train)
        
        #fit best model on training set
        best_model = result.best_estimator_
        
        # predictions on test set
        yhat = best_model.predict(X_test)
        
        #store yhat and test index for an outer fold 
        yhat_list.append(yhat)
        ind.append(test_ind)
        
        count += 1
        print("fold "+str(count))
        
    yhat_list = np.concatenate(yhat_list)
    ind = np.concatenate(ind)
    pred = [a for b, a in sorted(zip(ind, yhat_list))]
    pred = pd.Series(pred)
    
    return pred

#### SVM

In [ ]:
%%time
svm_raw_yhat = nested_cv(data, labels, "svm", {"C": [1,10], "gamma": [1,0.1, 0.01, 0.001, "auto"]})

In [ ]:
#compute accuracy and F1-score
%%time
svm_raw_accuracy = accuracy_score(labels, svm_raw_yhat)
svm_raw_f1 = f1_score(labels, svm_raw_yhat, average="macro")
(svm_raw_accuracy, svm_raw_f1)

In [ ]:
#save accuracy and F1-score
with open('v2_output/svm_raw_f1_f1.pkl', 'wb') as f:
    pickle.dump(svm_raw_f1, f)

In [ ]:
#save predictions
with open('v2_output/svm_raw_yhat_f1.pkl', 'wb') as f:
    pickle.dump(svm_raw_yhat, f)

#### KNN

In [ ]:
%%time
knn_raw_yhat = nested_cv(data, labels, "knn", {'n_neighbors': [2,5,15]})

In [ ]:
#%%time
knn_raw_accuracy = accuracy_score(labels, knn_raw_yhat)
knn_raw_f1 = f1_score(labels, knn_raw_yhat, average="macro")
(knn_raw_accuracy, knn_raw_f1)

In [ ]:
with open('v2_output/knn_raw_acc.pkl', 'wb') as f:
    pickle.dump(knn_raw_accuracy, f)

In [ ]:
with open('v2_output/knn_raw_yhat_acc.pkl', 'wb') as f:
    pickle.dump(knn_raw_yhat, f)

#### RF

In [ ]:
%%time
rf_raw_yhat = nested_cv(data, labels, "rf", {'max_features': ["sqrt", "log2"]})

In [ ]:
#%%time
rf_raw_accuracy = accuracy_score(labels, rf_raw_yhat)
rf_raw_f1 = f1_score(labels, rf_raw_yhat, average="macro")
(rf_raw_accuracy, rf_raw_f1)

In [ ]:
with open('v2_output/rf_raw_acc.pkl', 'wb') as f:
    pickle.dump(rf_raw_accuracy, f)

In [ ]:
with open('v2_output/rf_raw_yhat_acc.pkl', 'wb') as f:
    pickle.dump(rf_raw_yhat, f)

#### MLR

In [ ]:
%%time
mlr_raw_yhat = nested_cv(data, labels, "mlr", {'penalty': ["none", "l2"]})

In [ ]:
%%time
mlr_raw_accuracy = accuracy_score(labels, mlr_raw_yhat)
mlr_raw_f1 = f1_score(labels, mlr_raw_yhat, average="macro")
(mlr_raw_accuracy, mlr_raw_f1)

In [ ]:
with open('v2_output/mlr_raw_f1_f1.pkl', 'wb') as f:
    pickle.dump(mlr_raw_f1, f)

In [ ]:
with open('v2_output/mlr_raw_yhat_f1.pkl', 'wb') as f:
    pickle.dump(mlr_raw_yhat, f)

#### LDA

In [ ]:
#equal priors
priors = [1/7]*7

In [ ]:
%%time
lda_raw_yhat = nested_cv(data, labels, "lda", {'priors': [None,priors]})

In [ ]:
%%time
lda_raw_accuracy = accuracy_score(labels, lda_raw_yhat)
lda_raw_f1 = f1_score(labels, lda_raw_yhat, average="macro")
(lda_raw_accuracy, lda_raw_f1)

In [ ]:
with open('v2_output/lda_raw_f1_f1.pkl', 'wb') as f:
    pickle.dump(lda_raw_f1, f)

In [ ]:
with open('v2_output/lda_raw_yhat_f1.pkl', 'wb') as f:
    pickle.dump(lda_raw_yhat, f)

#### QDA

In [ ]:
%%time
qda_raw_yhat = nested_cv(data, labels, "qda", {'priors': [None,priors]})

In [ ]:
%%time
qda_raw_accuracy = accuracy_score(labels, qda_raw_yhat)
qda_raw_f1 = f1_score(labels, qda_raw_yhat, average="macro")
(qda_raw_accuracy, qda_raw_f1)

In [ ]:
with open('v2_output/qda_raw_f1_f1.pkl', 'wb') as f:
    pickle.dump(qda_raw_f1, f)

In [ ]:
with open('v2_output/qda_raw_yhat_f1.pkl', 'wb') as f:
    pickle.dump(qda_raw_yhat, f)

### Nested cross-validation on PCA embedding

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
#in: X (array), y (pd Series), classifier (string) , para_grid (dict)
#function stores accuracy and F1-scores in acc and f1 respectively

def nested_cv_pca(X, y, dimensions, classifier, param_grid, acc, f1, outer_folds=3, inner_folds=3):
    
    #empty list for test index and yhat
    ind = [] 
    yhat_list = []

    
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=1)
    
    for train_ind, test_ind in outer_cv.split(X):
         
        warnings.filterwarnings("ignore")
        
        #split data into train and test
        X_train, X_test = X[train_ind, :], X[test_ind, :]
        y_train, y_test = y[train_ind], y[test_ind]
        
        #select classifier
        if classifier == "svm":
            model_to_tune = SVC(random_state=1)
        if classifier == "knn":
            model_to_tune = KNeighborsClassifier()
        if classifier == "rf":
            model_to_tune = RandomForestClassifier(random_state=1)            
        if classifier == "mlr":
            model_to_tune = LogisticRegression(random_state=1)
        if classifier == "lda":
            model_to_tune = LinearDiscriminantAnalysis()
        if classifier == "qda":
            model_to_tune = QuadraticDiscriminantAnalysis()
            
        
        #specify pca model
        pca_model = PCA(n_components=dimensions)

        #inner cross-validation
        inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=1)
        
        steps = [('pca', pca_model), ('classifier', model_to_tune)]
        pipeline = Pipeline(steps)
        
        grid = GridSearchCV(pipeline, param_grid, cv=inner_cv, refit=True, n_jobs=-1, scoring='accuracy')
        result = grid.fit(X_train, y_train)
        
        #fit best model on training set
        best_model = result.best_estimator_
        
        # predictions on test set
        yhat = best_model.predict(X_test)
        
        #store yhat and test index for an outer fold 
        yhat_list.append(yhat)
        ind.append(test_ind)        
        
    yhat_list = np.concatenate(yhat_list)
    ind = np.concatenate(ind)
    pred = [a for b, a in sorted(zip(ind, yhat_list))]
    pred = pd.Series(pred)
    
    acc.append(accuracy_score(labels, pred))
    f1.append(f1_score(labels, pred, average="macro"))   

In [ ]:
pca_dim = [1,2,10,25,40]

#### SVM

In [ ]:
#create empty list for accuracy and f1-scores
pca_acc = []
pca_f1 = []

In [ ]:
%%time
for i in range(len(pca_dim)):
    nested_cv_pca(data, labels, pca_dim[i],
                   "svm", {'classifier__C': [1,10], 'classifier__gamma': [1,0.1, 0.01, 0.001, "auto"] }, 
                   pca_acc, pca_f1)



In [ ]:
with open('v2_output/svm_pca_accuracy.pkl', 'wb') as f:
    pickle.dump(pca_acc, f)

In [ ]:
with open('v2_output/svm_pca_f1_f1.pkl', 'wb') as f:
    pickle.dump(pca_f1, f)

#### KNN

In [ ]:
pca_acc = []
pca_f1 = []

In [ ]:
%%time
for i in range(len(pca_dim)):
    nested_cv_pca(data, labels, pca_dim[i],
                   "knn", {'classifier__n_neighbors': [2,5,15]}, 
                   pca_acc, pca_f1)

In [ ]:
with open('v2_output/knn_pca_acc.pkl', 'wb') as f:
    pickle.dump(pca_acc, f)

In [ ]:
with open('v2_output/knn_pca_f1_f1.pkl', 'wb') as f:
    pickle.dump(pca_f1, f)

#### RF

In [ ]:
pca_acc = []
pca_f1 = []

In [ ]:
%%time
for i in range(len(pca_dim)):
    nested_cv_pca(data, labels, pca_dim[i],
                   "rf", {'classifier__max_features': ["sqrt", "log2"]}, 
                   pca_acc, pca_f1)

In [ ]:
with open('v2_output/rf_pca_acc.pkl', 'wb') as f:
    pickle.dump(pca_acc, f)

In [ ]:
with open('v2_output/rf_pca_f1_f1.pkl', 'wb') as f:
    pickle.dump(pca_f1, f)

#### MLR

In [ ]:
pca_acc = []
pca_f1 = []

In [ ]:
%%time
for i in range(len(pca_dim)):
    nested_cv_pca(data, labels, pca_dim[i],
                   "mlr", {'classifier__penalty': ["none", "l2"]}, 
                   pca_acc, pca_f1)

In [ ]:
with open('v2_output/mlr_pca_accuracy.pkl', 'wb') as f:
    pickle.dump(pca_acc, f)

In [ ]:
with open('v2_output/mlr_pca_f1_f1.pkl', 'wb') as f:
    pickle.dump(pca_f1, f)

#### LDA

In [ ]:
pca_acc = []
pca_f1 = []

In [ ]:
%%time
for i in range(len(pca_dim)):
    nested_cv_pca(data, labels, pca_dim[i],
                   "lda", {'classifier__priors': [None,priors]}, 
                   pca_acc, pca_f1)

In [ ]:
with open('v2_output/lda_pca_accuracy.pkl', 'wb') as f:
    pickle.dump(pca_acc, f)

In [ ]:
with open('v2_output/lda_pca_f1_f1.pkl', 'wb') as f:
    pickle.dump(pca_f1, f)

#### QDA

In [ ]:
priors = [1/7]*7
priors = np.array(priors)
priors

In [ ]:
pca_acc = []
pca_f1 = []

In [ ]:
%%time
for i in range(len(pca_dim)):
    nested_cv_pca(data, labels, pca_dim[i],
                   "qda", {'classifier__priors': [None,priors]}, 
                   pca_acc, pca_f1)

In [ ]:
with open('v2_output/qda_pca_accuracy.pkl', 'wb') as f:
    pickle.dump(pca_acc, f)

In [ ]:
with open('v2_output/qda_pca_f1_f1.pkl', 'wb') as f:
    pickle.dump(pca_f1, f)

### Nested cross-validation on UMAP embedding

In [ ]:
import random
random.seed(2678136)

#Generate 20 random numbers and list them
seed_list = random.sample(range(10**0, 10**6), 20)

#for nested cross-validation only one seed is used
seed=[seed_list[0]]

In [ ]:
dim=[1,2,10,25,40]
neigh=[2, 5, 15, 25, 50, 75, 100]

In [ ]:
#create combinations of number of dimensions and number of neighbors 
a = [dim,neigh,seed]
test_paramters=list(itertools.product(*a))

In [ ]:
test_paramters

In [ ]:
#nested cross-validation
#in: X (array), y (pd Series), classifier (string) , para_grid (dict)
#function stores predictions (dict_yhat) and most optimal parameters (dict_param)

def nested_cv_umap(X, y, dimensions, neigbors, seed, classifier, param_grid, dict_yhat, dict_param,
                   outer_folds=3, inner_folds=3):
    
    #empty list for test index and yhat
    ind = [] 
    yhat_list = []
    best_param = {}
    count = 1
    
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=1)
    
    for train_ind, test_ind in outer_cv.split(X):
         
        
        warnings.filterwarnings("ignore")
        
        #split data into train and test
        X_train, X_test = X[train_ind, :], X[test_ind, :]
        y_train, y_test = y[train_ind], y[test_ind]
        
        #select classifier
        if classifier == "svm":
            model_to_tune = SVC(random_state=1)
        if classifier == "knn":
            model_to_tune = KNeighborsClassifier()
        if classifier == "rf":
            model_to_tune = RandomForestClassifier(random_state=1)
        if classifier == "mlr":
            model_to_tune = LogisticRegression(random_state=1)
        if classifier == "lda":
            model_to_tune = LinearDiscriminantAnalysis()
        if classifier == "qda":
            model_to_tune = QuadraticDiscriminantAnalysis()
            
        
        #specify umap model
        umap_model = umap.UMAP(n_components=dimensions, n_neighbors=neigbors, random_state=seed)

        #inner cross-validation
        inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=1)
        
        steps = [('umap', umap_model), ('classifier', model_to_tune)]
        pipeline = Pipeline(steps)
        
        grid = GridSearchCV(pipeline, param_grid, cv=inner_cv, refit=True, n_jobs=-1, scoring='accuracy')
        result = grid.fit(X_train, y_train)
        
        #fit best model on training set
        best_model = result.best_estimator_
        
        # predictions on test set
        yhat = best_model.predict(X_test)
        
        #store yhat and test index for an outer fold 
        yhat_list.append(yhat)
        ind.append(test_ind)
        
        #store best model
        best_param[str(count)] = best_model
        
        count += 1
        
        
    yhat_list = np.concatenate(yhat_list)
    ind = np.concatenate(ind)
    pred = [a for b, a in sorted(zip(ind, yhat_list))]
    pred = pd.Series(pred)
    
    key = "d"+str(dimensions)+"n"+str(neigbors)
    dict_yhat[key] = pred
    
    dict_param[key] = best_param
    
    print(key)
    

#### SVM

In [ ]:
#create empty list for predictions and best parameters
dict_yhat = {}
dict_param = {}

In [ ]:
#run function for every hyperparameter combination
%%time
for i in range(len(test_paramters)):
    nested_cv_umap(data, labels, test_paramters[i][0], test_paramters[i][1], test_paramters[i][2], 
                   "svm", {'classifier__C': [1,10], 'classifier__gamma': [1,0.1, 0.01, 0.001, "auto"] }, 
                   dict_yhat, dict_param)


In [ ]:
#save dictionary with predictions for every hyperparameter combination
with open('v2_output/svm_umap_yhat_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_yhat, f)

In [ ]:
#create dictionary with most optimal parameters in each fold
test_keys = list(dict_param.keys())
dict_params = {}

for i in range(len(test_keys)):
    dict_params_inner = {}

    for j in range(1,4):
        dict_params_inner[str(j)] = dict_param[str(test_keys[i])][str(j)][1]
        
    dict_params[str(test_keys[i])] =  dict_params_inner

In [ ]:
with open('v2_output/svm_umap_params_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_params, f)

#### KNN

In [ ]:
dict_yhat = {}
dict_param = {}

In [ ]:
%%time
for i in range(len(test_paramters)):
    nested_cv_umap(data, labels, test_paramters[i][0], test_paramters[i][1], test_paramters[i][2],
                   "knn", {'classifier__n_neighbors': [2,5,15]}, dict_yhat, dict_param)


In [ ]:
with open('v2_output/knn_umap_yhat_0_acc.pkl', 'wb') as f:
    pickle.dump(dict_yhat, f)

In [ ]:
test_keys = list(dict_param.keys())
dict_params = {}

for i in range(len(test_keys)):
    dict_params_inner = {}

    for j in range(1,4):
        dict_params_inner[str(j)] = dict_param[str(test_keys[i])][str(j)][1]
        
    dict_params[str(test_keys[i])] =  dict_params_inner

In [ ]:
with open('v2_output/knn_umap_params_0_acc.pkl', 'wb') as f:
    pickle.dump(dict_params, f)

#### RF

In [ ]:
dict_yhat = {}
dict_param = {}

In [ ]:
%%time
for i in range(len(test_paramters)):
    nested_cv_umap(data, labels, test_paramters[i][0], test_paramters[i][1], test_paramters[i][2],
                   "rf", {'classifier__max_features': ["sqrt", "log2"]}, dict_yhat, dict_param)

In [ ]:
with open('v2_output/rf_umap_yhat_0_acc.pkl', 'wb') as f:
    pickle.dump(dict_yhat, f)

In [ ]:
test_keys = list(dict_param.keys())
dict_params = {}

for i in range(len(test_keys)):
    dict_params_inner = {}

    for j in range(1,4):
        dict_params_inner[str(j)] = dict_param[str(test_keys[i])][str(j)][1]
        
    dict_params[str(test_keys[i])] =  dict_params_inner

In [ ]:
with open('v2_output/rf_umap_params_0_acc.pkl', 'wb') as f:
    pickle.dump(dict_params, f)

#### MLR

In [ ]:
dict_yhat = {}
dict_param = {}

In [ ]:
%%time
for i in range(len(test_paramters)):
    nested_cv_umap(data, labels, test_paramters[i][0], test_paramters[i][1], test_paramters[i][2],
                   "mlr", {'classifier__penalty': ["none", "l2"]}, dict_yhat, dict_param)

In [ ]:
with open('v2_output/mlr_umap_yhat_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_yhat, f)

In [ ]:
test_keys = list(dict_param.keys())
dict_params = {}

for i in range(len(test_keys)):
    dict_params_inner = {}

    for j in range(1,4):
        dict_params_inner[str(j)] = dict_param[str(test_keys[i])][str(j)][1]
        
    dict_params[str(test_keys[i])] =  dict_params_inner

In [ ]:
with open('v2_output/mlr_umap_params_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_params, f)

#### LDA

In [ ]:
dict_yhat = {}
dict_param = {}

In [ ]:
%%time
for i in range(len(test_paramters)):
    nested_cv_umap(data, labels, test_paramters[i][0], test_paramters[i][1],
                       test_paramters[i][2], "lda", {'classifier__priors': [None,priors]}, dict_yhat, dict_param)

In [ ]:
with open('v2_output/lda_umap_yhat_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_yhat, f)

In [ ]:
test_keys = list(dict_param.keys())
dict_params = {}

for i in range(len(test_keys)):
    dict_params_inner = {}

    for j in range(1,4):
        dict_params_inner[str(j)] = dict_param[str(test_keys[i])][str(j)][1]
        
    dict_params[str(test_keys[i])] =  dict_params_inner

In [ ]:
with open('v2_output/lda_umap_params_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_params, f)

#### QDA

In [ ]:
priors = [1/7]*7
priors = np.array(priors)
priors

In [ ]:
dict_yhat = {}
dict_param = {}

In [ ]:
%%time
for i in range(len(test_paramters)):
    nested_cv_umap(data, labels, test_paramters[i][0], test_paramters[i][1],
                       test_paramters[i][2], "qda", {'classifier__priors': [None,priors]}, dict_yhat, dict_param)

In [ ]:
with open('v2_output/qda_umap_yhat_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_yhat, f)

In [ ]:
test_keys = list(dict_param.keys())
dict_params = {}

for i in range(len(test_keys)):
    dict_params_inner = {}

    for j in range(1,4):
        dict_params_inner[str(j)] = dict_param[str(test_keys[i])][str(j)][1]
        
    dict_params[str(test_keys[i])] =  dict_params_inner

In [ ]:
with open('v2_output/qda_umap_params_0_f1.pkl', 'wb') as f:
    pickle.dump(dict_params, f)

#### Compute accuracy and F1-score

In [ ]:
#load dictionary with predictions
with open('v2_output/knn_umap_yhat_0_acc.pkl', 'rb') as f:
    dict_yhat = pickle.load(f)

In [ ]:
dict_yhat

In [ ]:
dict_acc = {}
dict_f1 = {}

In [ ]:
#function computes accuracy and f1-scores and store them in dict_acc and dict_f1
def compute_dict_scores(key, dict_yhat, dict_acc, dict_f1, labels ):
    
    dict_acc[key] = accuracy_score(labels, dict_yhat[key])
    dict_f1[key] = f1_score(labels, dict_yhat[key], average="macro")
    

In [ ]:
test_parameters_accuracy = list(dict_yhat.keys())

In [ ]:
#run function for every hyperparameter combination
for i in range(len(test_parameters_accuracy)):
    compute_dict_scores(test_parameters_accuracy[i], dict_yhat, dict_acc, dict_f1, labels)

In [ ]:
dict_acc

In [ ]:
#save dictionary with accuracy/f1 score for every hyperparameter combination (key)
with open('v2_output/knn_umap_acc_opt.pkl', 'wb') as f:
    pickle.dump(dict_acc, f)

### Plot nested CV results

In [ ]:
#load dictionary with accuracy/f1 score for every hyperparameter combination (key)
with open('v2_output/lda_umap_f1_f1.pkl', 'rb') as f:
    dict_ = pickle.load(f)

In [ ]:
def create_table(scores):
    values = {
        'n2': [scores['d1n2'], scores['d2n2'], scores['d10n2'], scores['d25n2'], scores['d40n2']],
        'n5': [scores['d1n5'], scores['d2n5'], scores['d10n5'], scores['d25n5'], scores['d40n5']],
        'n15': [scores['d1n15'], scores['d2n15'], scores['d10n15'], scores['d25n15'], scores['d40n15']],
        'n25': [scores['d1n25'], scores['d2n25'], scores['d10n25'], scores['d25n25'], scores['d40n25']],
        'n50': [scores['d1n50'], scores['d2n50'], scores['d10n50'],scores['d25n50'], scores['d40n50']],
        'n75': [scores['d1n75'], scores['d2n75'], scores['d10n75'], scores['d25n75'], scores['d40n75']],
        'n100': [scores['d1n100'], scores['d2n100'], scores['d10n100'], scores['d25n100'], scores['d40n100']]
                }
    
    table = pd.DataFrame(values, index=['d1','d2','d10','d25','d40'])

    return table

In [ ]:
#table with accuracy/f1-score for umap (dimensions xneighbors)
table_acc = create_table(dict_)
table_acc

In [ ]:
#plots accuracy/f1 score for umap, pca and unreduced in one figure
def plot(table, accuracy_raw, accuracy_pca, name, path):
    
    t=np.array([2, 5, 15, 25, 50, 75, 100])
    #labels=['d1','d2','d10', 'd25', 'd40', 'raw']
    labels=['d1','d2','d10', 'd25', 'd40', 'pca1','pca2','pca10', 'pca25', 'pca40', 'raw']
    
    plt.figure(figsize = (7,4) )
    ax = plt.gca()
    ax.plot(t, table.iloc[0,:], color = 'red', label=labels[0], lw=1.5, marker='.')
    
    ax.plot(t, table.iloc[1,:], color = 'orange', label=labels[1], lw=1.5, marker='.')
    
    ax.plot(t, table.iloc[2,:], color = 'green', label=labels[2], lw=1.5, marker='.')
    
    ax.plot(t, table.iloc[3,:], color = 'dodgerblue', label=labels[3], lw=1.5, marker='.')
    
    ax.plot(t, table.iloc[4,:], color = 'violet', label=labels[4], lw=1.5, marker='.')
    
    plt.axhline(y=accuracy_pca[0], color='red', linestyle='--', label=labels[5])
    plt.axhline(y=accuracy_pca[1], color='orange', linestyle='--', label=labels[6])
    plt.axhline(y=accuracy_pca[2], color='green', linestyle='--', label=labels[7])
    plt.axhline(y=accuracy_pca[3], color='dodgerblue', linestyle='--', label=labels[8])
    plt.axhline(y=accuracy_pca[4], color='violet', linestyle='--', label=labels[9])
    
    plt.axhline(y=accuracy_raw, color='gray', linestyle='-.', label=labels[10])
    
    plt.ylim(0.5, 1.01)
    #plt.ylim(0, 1.01)
    
    plt.xlabel("Number of neigbors")
    #plt.ylabel("Accuracy")
    plt.ylabel("F1 score")
    
    plt.savefig(path, facecolor='white', bbox_inches='tight', dpi=300)

In [ ]:
#load accuracy/f1 score for unreduced dataset
with open('v2_output/lda_raw_f1_f1.pkl', 'rb') as f:
    raw = pickle.load(f)

In [ ]:
#load accuracy/f1 score for PCA embeddings
with open('v2_output/lda_pca_f1_f1.pkl', 'rb') as f:
    pca = pickle.load(f)

In [ ]:
plot(table_acc, raw, pca, '-', 'v2_output/lda_plot_f1_f1_w')

## Non cross-validated analysis

### Split into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=.2, random_state=2022)

### Performance on unreduced dataset

In [ ]:
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(x_train, y_train)
knn_raw_yhat = knn.predict(x_test)

In [ ]:
#compute accuracy and F1-score
knn_raw_accuracy = accuracy_score(y_test, knn_raw_yhat)
knn_raw_f1 = f1_score(y_test, knn_raw_yhat, average="macro")
(knn_raw_accuracy, knn_raw_f1)

In [ ]:
with open('v2_output/knn100_raw_accuracy.pkl', 'wb') as f:
    pickle.dump(knn_raw_accuracy, f)
    
with open('v2_output/knn100_raw_f1.pkl', 'wb') as f:
    pickle.dump(knn_raw_f1, f)

In [ ]:
with open('v2_output/knn100_raw_yhat.pkl', 'wb') as f:
    pickle.dump(knn_raw_yhat, f)

### Performance on UMAP embeddings

#### Compute embeddings

In [ ]:
#compute training and test embeddings
#embeddings are stored in dict_emb

def compute_embeddings(x_train, x_test, dimensions, neigbors, seed, dict_emb):
    
    
    embedder = umap.UMAP(n_neighbors=neigbors, n_components=dimensions, random_state=seed).fit(x_train)
    
    train_embedding = embedder.embedding_
    
    test_embedding = embedder.transform(x_test)
    
    key = "d"+str(dimensions)+"n"+str(neigbors)
    
    dict_emb[key] = (train_embedding, test_embedding)
    
    
    print(key)
    

In [ ]:
#use the same list with random numbers as before
seed_list

In [ ]:
dim=[1,2,10,25,40]
neigh=[2, 5, 15, 25, 50, 75, 100]
#test=[1]
seed=[seed_list[9]]
seed

In [ ]:
#create combinations of number of dimensions and number of neighbors 
#repeat for 10 different random seeds
a = [dim,neigh,seed]
test_paramters=list(itertools.product(*a))

In [ ]:
test_paramters

In [ ]:
#create empty dictionary to store embeddings
dict_emb = {}

In [ ]:
#repeat for 10 different random seeds
%%time
for i in range(len(test_paramters)):
    compute_embeddings(x_train, x_test, test_paramters[i][0], test_paramters[i][1],
                       test_paramters[i][2], dict_emb)

In [ ]:
#save dictionary which contains a training embedding and test embeddind for every hyperparameter combination
#repeat for 10 different seeds
with open('v2_output/Chu_emb_9.pkl', 'wb') as f:
    pickle.dump(dict_emb, f)

#### Compute accuracy and F1-score

In [ ]:
#create empty dictionaries to store accuracy and F1 scores
dict_knn2_acc = {}
dict_knn5_acc = {}
dict_knn15_acc = {}
dict_knn25_acc = {}
dict_knn50_acc = {}
dict_knn75_acc = {}
dict_knn100_acc = {}

dict_knn2_f1 = {}
dict_knn5_f1 = {}
dict_knn15_f1 = {}
dict_knn25_f1 = {}
dict_knn50_f1 = {}
dict_knn75_f1 = {}
dict_knn100_f1 = {}

In [ ]:
#fit classification models and compute accuracy and f1 scores

def compute_accuracy_scores(key, train_embedding, test_embedding, y_train, y_test, 
                            knn5_acc, knn15_acc, 
                            knn25_acc, knn50_acc, knn75_acc, knn100_acc,
                            knn2_f1, knn5_f1, knn15_f1, 
                            knn25_f1, knn50_f1, knn75_f1, knn100_f1):
    

    from sklearn.neighbors import KNeighborsClassifier
    
    #fit classifiers

    knn2_umap = KNeighborsClassifier(n_neighbors=2)
    knn2_umap.fit(train_embedding, y_train)
    knn2_yhat = knn2_umap.predict(test_embedding)
    
    knn5_umap = KNeighborsClassifier(n_neighbors=5)
    knn5_umap.fit(train_embedding, y_train)
    knn5_yhat = knn5_umap.predict(test_embedding)
    
    knn15_umap = KNeighborsClassifier(n_neighbors=15)
    knn15_umap.fit(train_embedding, y_train)
    knn15_yhat = knn15_umap.predict(test_embedding)
    
    knn25_umap = KNeighborsClassifier(n_neighbors=25)
    knn25_umap.fit(train_embedding, y_train)
    knn25_yhat = knn25_umap.predict(test_embedding)
    
    knn50_umap = KNeighborsClassifier(n_neighbors=50)
    knn50_umap.fit(train_embedding, y_train)
    knn50_yhat = knn50_umap.predict(test_embedding)

    knn75_umap = KNeighborsClassifier(n_neighbors=75)
    knn75_umap.fit(train_embedding, y_train)
    knn75_yhat = knn75_umap.predict(test_embedding)

    knn100_umap = KNeighborsClassifier(n_neighbors=100)
    knn100_umap.fit(train_embedding, y_train)
    knn100_yhat = knn100_umap.predict(test_embedding)

 
    #compute scores
    

    knn2_acc[key] = [accuracy_score(y_test, knn2_yhat)]
    knn2_f1[key] = [f1_score(y_test, knn2_yhat, average="macro")]

    knn5_acc[key] = [accuracy_score(y_test, knn5_yhat)]
    knn5_f1[key] = [f1_score(y_test, knn5_yhat, average="macro")]

    knn15_acc[key] = [accuracy_score(y_test, knn15_yhat)]
    knn15_f1[key] = [f1_score(y_test, knn15_yhat, average="macro")]

    knn25_acc[key] = [accuracy_score(y_test, knn25_yhat)]
    knn25_f1[key] = [f1_score(y_test, knn25_yhat, average="macro")]

    knn50_acc[key] = [accuracy_score(y_test, knn50_yhat)]
    knn50_f1[key] = [f1_score(y_test, knn50_yhat, average="macro")]

    knn75_acc[key] = [accuracy_score(y_test, knn75_yhat)]
    knn75_f1[key] = [f1_score(y_test, knn75_yhat, average="macro")]

    knn100_acc[key] = [accuracy_score(y_test, knn100_yhat)]
    knn100_f1[key] = [f1_score(y_test, knn100_yhat, average="macro")]



In [ ]:
#fit classification models and compute accuracy and f1 scores

def compute_accuracy_scores(key, train_embedding, test_embedding, y_train, y_test, 
                            knn2_acc, knn5_acc, knn15_acc, 
                            knn25_acc, knn50_acc, knn75_acc, knn100_acc,
                            knn2_f1, knn5_f1, knn15_f1, 
                            knn25_f1, knn50_f1, knn75_f1, knn100_f1):
    

    from sklearn.neighbors import KNeighborsClassifier
    
    #fit classifiers

    
    knn2_umap = KNeighborsClassifier(n_neighbors=2)
    knn2_umap.fit(train_embedding, y_train)
    knn2_yhat = knn2_umap.predict(test_embedding)
    
    knn5_umap = KNeighborsClassifier(n_neighbors=5)
    knn5_umap.fit(train_embedding, y_train)
    knn5_yhat = knn5_umap.predict(test_embedding)
    
    knn15_umap = KNeighborsClassifier(n_neighbors=15)
    knn15_umap.fit(train_embedding, y_train)
    knn15_yhat = knn15_umap.predict(test_embedding)
    
    knn25_umap = KNeighborsClassifier(n_neighbors=25)
    knn25_umap.fit(train_embedding, y_train)
    knn25_yhat = knn25_umap.predict(test_embedding)
    
    knn50_umap = KNeighborsClassifier(n_neighbors=50)
    knn50_umap.fit(train_embedding, y_train)
    knn50_yhat = knn50_umap.predict(test_embedding)

    knn75_umap = KNeighborsClassifier(n_neighbors=75)
    knn75_umap.fit(train_embedding, y_train)
    knn75_yhat = knn75_umap.predict(test_embedding)

    knn100_umap = KNeighborsClassifier(n_neighbors=100)
    knn100_umap.fit(train_embedding, y_train)
    knn100_yhat = knn100_umap.predict(test_embedding)

 
    #compute scores


    knn2_acc[key].append(accuracy_score(y_test, knn2_yhat))
    knn2_f1[key].append(f1_score(y_test, knn2_yhat, average="macro"))

    knn5_acc[key].append(accuracy_score(y_test, knn5_yhat))
    knn5_f1[key].append(f1_score(y_test, knn5_yhat, average="macro"))

    knn15_acc[key].append(accuracy_score(y_test, knn15_yhat))
    knn15_f1[key].append(f1_score(y_test, knn15_yhat, average="macro"))

    knn25_acc[key].append(accuracy_score(y_test, knn25_yhat))
    knn25_f1[key].append(f1_score(y_test, knn25_yhat, average="macro"))

    knn50_acc[key].append(accuracy_score(y_test, knn50_yhat))
    knn50_f1[key].append(f1_score(y_test, knn50_yhat, average="macro"))

    knn75_acc[key].append(accuracy_score(y_test, knn75_yhat))
    knn75_f1[key].append(f1_score(y_test, knn75_yhat, average="macro"))

    knn100_acc[key].append(accuracy_score(y_test, knn100_yhat))
    knn100_f1[key].append(f1_score(y_test, knn100_yhat, average="macro"))

In [ ]:
with open('v2_output/Chu_emb_9.pkl', 'rb') as f:
    dict_emb = pickle.load(f)

In [ ]:
#run function for every hyperparameter combinations
for i in range(len(test_parameters_accuracy)):
    compute_accuracy_scores(test_parameters_accuracy[i], dict_emb[test_parameters_accuracy[i]][0], 
                       dict_emb[test_parameters_accuracy[i]][1], 
                       y_train, y_test, 
                       dict_knn2_acc, dict_knn5_acc, dict_knn15_acc, 
                        dict_knn25_acc, dict_knn50_acc, dict_knn75_acc, dict_knn100_acc,
                        dict_knn2_f1, dict_knn5_f1, dict_knn15_f1, 
                        dict_knn25_f1, dict_knn50_f1, dict_knn75_f1, dict_knn100_f1)

In [ ]:
#save dictionaries which stores 10 performance scores for each hyperparameter combination
with open('v2_output/knn2_umap_acc.pkl', 'wb') as f:
    pickle.dump(dict_knn2_acc, f)
with open('v2_output/knn2_umap_f1.pkl', 'wb') as f:
    pickle.dump(dict_knn2_f1, f)
    
with open('v2_output/knn5_umap_acc.pkl', 'wb') as f:
    pickle.dump(dict_knn5_acc, f)
with open('v2_output/knn5_umap_f1.pkl', 'wb') as f:
    pickle.dump(dict_knn5_f1, f)
    
with open('v2_output/knn15_umap_acc.pkl', 'wb') as f:
    pickle.dump(dict_knn15_acc, f)
with open('v2_output/knn15_umap_f1.pkl', 'wb') as f:
    pickle.dump(dict_knn15_f1, f)
    
with open('v2_output/knn25_umap_acc.pkl', 'wb') as f:
    pickle.dump(dict_knn25_acc, f)
with open('v2_output/knn25_umap_f1.pkl', 'wb') as f:
    pickle.dump(dict_knn25_f1, f)
    
with open('v2_output/knn50_umap_acc.pkl', 'wb') as f:
    pickle.dump(dict_knn50_acc, f)
with open('v2_output/knn50_umap_f1.pkl', 'wb') as f:
    pickle.dump(dict_knn50_f1, f)
    
with open('v2_output/knn75_umap_acc.pkl', 'wb') as f:
    pickle.dump(dict_knn75_acc, f)
with open('v2_output/knn75_umap_f1.pkl', 'wb') as f:
    pickle.dump(dict_knn75_f1, f)
    
with open('v2_output/knn100_umap_acc.pkl', 'wb') as f:
    pickle.dump(dict_knn100_acc, f)
with open('v2_output/knn100_umap_f1.pkl', 'wb') as f:
    pickle.dump(dict_knn100_f1, f)


### Plot results

In [ ]:
with open('v2_output/qda_umap_acc.pkl', 'rb') as f:
    dict_acc = pickle.load(f)

with open('v2_output/qda_umap_f1.pkl', 'rb') as f:
    dict_f1 = pickle.load(f)

In [ ]:
df_acc = pd.DataFrame(dict_acc)

df_f1 = pd.DataFrame(dict_f1)

In [ ]:
dict_acc_mean = dict(df_acc.describe().loc['mean'])
dict_acc_50 = dict(df_acc.describe().loc['50%'])
dict_acc_25 = dict(df_acc.describe().loc['25%'])
dict_acc_75 = dict(df_acc.describe().loc['75%'])

dict_f1_mean = dict(df_f1.describe().loc['mean'])
dict_f1_50 = dict(df_f1.describe().loc['50%'])
dict_f1_25 = dict(df_f1.describe().loc['25%'])
dict_f1_75 = dict(df_f1.describe().loc['75%'])

In [ ]:
def create_table(scores):
    values = {
        'n2': [scores['d1n2'], scores['d2n2'], scores['d10n2'], scores['d25n2'], scores['d40n2']],
        'n5': [scores['d1n5'], scores['d2n5'], scores['d10n5'], scores['d25n5'], scores['d40n5']],
        'n15': [scores['d1n15'], scores['d2n15'], scores['d10n15'], scores['d25n15'], scores['d40n15']],
        'n25': [scores['d1n25'], scores['d2n25'], scores['d10n25'], scores['d25n25'], scores['d40n25']],
        'n50': [scores['d1n50'], scores['d2n50'], scores['d10n50'],scores['d25n50'], scores['d40n50']],
        'n75': [scores['d1n75'], scores['d2n75'], scores['d10n75'], scores['d25n75'], scores['d40n75']],
        'n100': [scores['d1n100'], scores['d2n100'], scores['d10n100'], scores['d25n100'], scores['d40n100']]
                }
    
    table = pd.DataFrame(values, index=['d1','d2','d10','d25','d40'])

    return table

In [ ]:
table_acc_mean = create_table(dict_acc_mean)
table_acc_50 = create_table(dict_acc_50)
table_acc_25 = create_table(dict_acc_25)
table_acc_75 = create_table(dict_acc_75)

table_f1_mean = create_table(dict_f1_mean)
table_f1_50 = create_table(dict_f1_50)
table_f1_25 = create_table(dict_f1_25)
table_f1_75 = create_table(dict_f1_75)

In [ ]:
def plot_iqr_knn(table_50, table_25, table_75, accuracy_raw, path):
    
    t=np.array([1, 2, 10, 25, 40])
    labels=['n2','n5','n15', 'n25', 'n50', 'n75', 'n100', 'raw']
    
    
    ax = plt.gca()
    ax.plot(t, table_50.iloc[0,:], color = 'red', label=labels[0], lw=1.5, marker='.')
    ax.fill_between(t, table_25.iloc[0,:], table_75.iloc[0,:], color = 'red', alpha=0.2, lw=0)
    
    ax.plot(t, table_50.iloc[1,:], color = 'orange', label=labels[1], lw=1.5, marker='.')
    ax.fill_between(t, table_25.iloc[1,:], table_75.iloc[1,:], color = 'orange', alpha=0.2, lw=0)
    
    ax.plot(t, table_50.iloc[2,:], color = 'green', label=labels[2], lw=1.5, marker='.')
    ax.fill_between(t, table_25.iloc[2,:], table_75.iloc[2,:], color = 'green', alpha=0.2, lw=0)
    
    ax.plot(t, table_50.iloc[3,:], color = 'dodgerblue', label=labels[3], lw=1.5, marker='.')
    ax.fill_between(t, table_25.iloc[3,:], table_75.iloc[3,:], color = 'dodgerblue', alpha=0.2, lw=0)
    
    ax.plot(t, table_50.iloc[4,:], color = 'violet', label=labels[4], lw=1.5, marker='.')
    ax.fill_between(t, table_25.iloc[4,:], table_75.iloc[4,:], color = 'violet', alpha=0.2, lw=0)
    
    ax.plot(t, table_50.iloc[5,:], color = 'slateblue', label=labels[5], lw=1.5, marker='.')
    ax.fill_between(t, table_25.iloc[5,:], table_75.iloc[5,:], color = 'slateblue', alpha=0.2, lw=0)
    
    ax.plot(t, table_50.iloc[6,:], color = 'brown', label=labels[6], lw=1.5, marker='.')
    ax.fill_between(t, table_25.iloc[6,:], table_75.iloc[6,:], color = 'brown', alpha=0.2, lw=0)
    
    
    plt.axhline(y=accuracy_raw, color='gray', linestyle='-.', label=labels[7])
    
    
    plt.ylim(0.5, 1.01)
    
    plt.xlabel("Number of dimensions")
    #plt.ylabel("Accuracy")
    plt.ylabel("F1 score")
    legend_outside = plt.legend(bbox_to_anchor=(1.20,0.72), loc='right')

    
    plt.savefig(path, facecolor='white', bbox_inches='tight', dpi=300)
    #plt.show()    
    

In [ ]:
with open('v2_output/knn5_raw_f1.pkl', 'rb') as f:
    raw = pickle.load(f)

In [ ]:
plot_iqr(table_f1_mean, table_f1_25, table_f1_75, 
         raw, 'v2_output/knn5_plot_f13')

## Figures

In [ ]:
y_testt = le.inverse_transform(y_test)

In [ ]:
with open('v2_output/pc2.pkl', 'rb') as f:
    pc = pickle.load(f)

In [ ]:
with open('v2_output/Chu_emb_0.pkl', 'rb') as f:
    emb = pickle.load(f)

In [ ]:
labelss = le.inverse_transform(labels)

In [ ]:
labels_legend = list(labels_copy.unique())
labels_legend

In [ ]:
pal = sns.color_palette()
color = pal.as_hex()

In [ ]:
color_dict = {}
keys = labels_legend
values = color
for i in range(len(keys)):
    color_dict[keys[i]] = values[i]
print(color_dict)

### training vs test data

In [ ]:
y_trainn = le.inverse_transform(y_train)

In [ ]:
#subplots

# create figure
fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharey=False)


# add subplots
#plt.figure(figsize = (10,10) )
sns.scatterplot(ax=axes[0], x=emb['d2n5'][0][:,0], y=emb['d2n5'][0][:,1], hue = y_trainn, palette=color_dict)
axes[0].axis('equal')
axes[0].set_title("Embedding of training data", fontsize=45)
axes[0].tick_params(axis='both', which='major', labelsize=22)

sns.scatterplot(ax=axes[1], x=emb['d2n5'][1][:,0], y=emb['d2n5'][1][:,1], hue = y_testt, palette=color_dict)
#axes[0].figure(figsize = (10,10) )
axes[1].axis('equal')
axes[1].set_title("Embedding of test data", fontsize=45)
axes[1].tick_params(axis='both', which='major', labelsize=22)


#hide legend in subplots
for ax in axes:
    ax.legend([],[], frameon=False)

    
# add legend
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center',  ncol=8, bbox_to_anchor=(0.5, -0.01), fontsize=22)


fig.tight_layout()

plt.savefig('v2_output/prediction_plots/train_vs_test1', facecolor='white', bbox_inches='tight', dpi=100)


plt.show()

### SVM

#### d2n5

In [ ]:
%%time
emb_full = umap.UMAP(n_neighbors=5, n_components=2, random_state=42653).fit(data)

In [ ]:
with open('v2_output/svm_umap_yhat_0_f1.pkl', 'rb') as f:
    svm_umap_yhat = pickle.load(f)

In [ ]:
svm_umap_yhatt = le.inverse_transform(svm_umap_yhat['d2n5'])

In [ ]:
#subplots

# create figure
fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharey=False)


# add subplots
#plt.figure(figsize = (10,10) )
sns.scatterplot(ax=axes[0], x=emb_full.embedding_[:,0], y=emb_full.embedding_[:,1], hue = labelss, palette=color_dict)
axes[0].axis('equal')
axes[0].set_title("True labels", fontsize=45)
axes[0].tick_params(axis='both', which='major', labelsize=22)

sns.scatterplot(ax=axes[1], x=emb_full.embedding_[:,0], y=emb_full.embedding_[:,1], hue = svm_umap_yhatt, palette=color_dict)
#axes[0].figure(figsize = (10,10) )
axes[1].axis('equal')
axes[1].set_title("Predicted labels", fontsize=45)
axes[1].tick_params(axis='both', which='major', labelsize=22)


#hide legend in subplots
for ax in axes:
    ax.legend([],[], frameon=False)

    
# add legend
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center',  ncol=8, bbox_to_anchor=(0.5, -0.01), fontsize=25)


fig.tight_layout()

plt.savefig('v2_output/prediction_plots/svm_umap_d2n5_opt1', facecolor='white', bbox_inches='tight', dpi=100)


plt.show()

#### d2n100

In [ ]:
%%time
emb_full = umap.UMAP(n_neighbors=100, n_components=2, random_state=42653).fit(data)

In [ ]:
svm_umap_yhatt = le.inverse_transform(svm_umap_yhat['d2n100'])

In [ ]:
#subplots

# create figure
fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharey=False)


# add subplots
#plt.figure(figsize = (10,10) )
sns.scatterplot(ax=axes[0], x=emb_full.embedding_[:,0], y=emb_full.embedding_[:,1], hue = labelss, palette=color_dict)
axes[0].axis('equal')
axes[0].set_title("True labels", fontsize=45)
axes[0].tick_params(axis='both', which='major', labelsize=22)

sns.scatterplot(ax=axes[1], x=emb_full.embedding_[:,0], y=emb_full.embedding_[:,1], hue = svm_umap_yhatt, palette=color_dict)
#axes[0].figure(figsize = (10,10) )
axes[1].axis('equal')
axes[1].set_title("Predicted labels", fontsize=45)
axes[1].tick_params(axis='both', which='major', labelsize=22)



#hide legend in subplots
for ax in axes:
    ax.legend([],[], frameon=False)

    
# add legend
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center',  ncol=8, bbox_to_anchor=(0.5, -0.01), fontsize=25)


fig.tight_layout()

plt.savefig('v2_output/prediction_plots/svm_umap_d2n100_opt1', facecolor='white', bbox_inches='tight', dpi=100)

plt.show()